<a href="https://colab.research.google.com/github/magladko/NAI/blob/master/mpp5/mpp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#mpp5 - Naiwny klasyfikator Bayesa
###@s22411

In [174]:
import pandas as pd

In [175]:
# available data options
options = {'price': ['vhigh', 'high', 'med', 'low'], 
           'maintenance': ['high', 'med', 'low'],
           'door_nr': [2, 3, 4, '5more'],
           'ppl_capacity': [2, 4, 'more'],
           'trunk_capacity': ['small', 'med', 'big'],
           'safety': ['low', 'med', 'high'],
           'def_decision': ['unacc', 'acc', 'good', 'vgood']}

In [176]:
train_df = pd.read_csv("https://raw.githubusercontent.com/magladko/NAI/master/mpp5/data/train", 
                       header=None, names=list(options.keys()))

test_df = pd.read_csv("https://raw.githubusercontent.com/magladko/NAI/master/mpp5/data/test", 
                       header=None, names=list(options.keys()))

In [177]:
#@title Czy wygładzać { run: "auto" }
smoothing = True #@param {type:"boolean"}


In [178]:
decisions = train_df["def_decision"].unique()

In [179]:
def bayes(row, smoothing, options, df):
    result = dict((decision, 1) for decision in options['def_decision'])    # result of the multiplications
    for decision in options['def_decision']:

        considered_df = df[df['def_decision'] == decision]  # only where decision occurs
        decision_count = len(considered_df)                             # number of records with considered decision
        
        # iterate through columns and values in considered row
        # print(len(row.keys()))
        for i in range(len(row.keys())):
            column = row.keys()[i]
            value = row[row.keys()[i]]

            # number of occurences with considered decision
            attr_occurences = len(considered_df[considered_df[column] == value])
 
            if smoothing:
                result[decision] *= (attr_occurences + 1) / (decision_count + len(options[column]))
            else:
                result[decision] *= attr_occurences / decision_count

        result[decision] *= decision_count / len(df)
    return result

In [180]:
correct_count = 0

for index, test_row in test_df.iterrows():
    result = bayes(test_row, smoothing, options, train_df)
    calculated_decison = max(result, key=result.get)
    correct_count += 1 if calculated_decison == test_row['def_decision'] else 0
    
    # print(result)

print("dokładność:", str(correct_count/len(test_df)*100) + '%')

dokładność: 100.0%


In [197]:
#@title Custom record { run: "auto" }
koszt_zakupu = "vhigh" #@param ["vhigh", "high", "med", "low"]
koszt_utrzymania = "high" #@param ["high", "med", "low"]
liczba_drzwi = "2" #@param ["2", "3", "4", "5more"]
ile_osob = "2" #@param ["2", "4", "more"]
bagaznik = "small" #@param ["small", "med", "big"]
bezpieczenstwo = "low" #@param ["low", "med", "high"]


custom_df = pd.DataFrame(data=[[koszt_zakupu, koszt_utrzymania, liczba_drzwi,
                                ile_osob, bagaznik, bezpieczenstwo, ""]], 
                         columns=list(options.keys()))


In [198]:
res_dict = bayes(custom_df.iloc[0], smoothing, options, train_df)
res = max(res_dict, key=res_dict.get)

print(res_dict)
print(res)

{'unacc': 1.0100223118898047e-06, 'acc': 1.1998456551479511e-11, 'good': 1.5364643592297637e-12, 'vgood': 1.312337285966582e-12}
unacc
